# Running Chatbot with Google Gemini on Colab

This notebook runs the chatbot project using Google's Gemini API on Google Colab.

## Step 1: Clone the GitHub repository

In [1]:
# Clone your GitHub repository
!git clone https://github.com/ST-SARAVANAPRIYAN/chatbot.git
%cd chatbot

Cloning into 'chatbot'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 45 (delta 11), reused 38 (delta 7), pack-reused 0 (from 0)
Receiving objects: 100% (45/45), 60.76 KiB | 10.13 MiB/s, done.
Resolving deltas: 100% (11/11), done.
/content/chatbot


## Step 2: Install required packages

In [2]:
# Install required packages
!pip install llama-index==0.12.33 llama-index-core llama-index-embeddings-gemini==0.1.0 chromadb python-dotenv streamlit spacy transformers torch langchain
!python -m spacy download en_core_web_sm

INFO: pip is looking at multiple versions of llama-index-embeddings-gemini to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install llama-index-core, llama-index-embeddings-gemini==0.1.0 and llama-index==0.12.33 because these package versions have conflicting dependencies.

The conflict is caused by:
    The user requested llama-index-core
    llama-index 0.12.33 depends on llama-index-core<0.13.0 and >=0.12.33
    llama-index-embeddings-gemini 0.1.0 depends on llama-index-core==0.10.0

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pip to attempt to solve the dependency conflict

ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 132.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the p

In [3]:
!pip install llama-index[embeddings-openai,vector-stores-chroma,llms-openai]


  Using cached llama_index_agent_openai-0.4.12-py3-none-any.whl.metadata (439 bytes)
  Using cached llama_index_cli-0.4.3-py3-none-any.whl.metadata (1.4 kB)
  Using cached llama_index_core-0.12.46-py3-none-any.whl.metadata (2.5 kB)
  Using cached llama_index_embeddings_openai-0.3.1-py3-none-any.whl.metadata (684 bytes)
  Using cached llama_index_indices_managed_llama_cloud-0.7.8-py3-none-any.whl.metadata (3.3 kB)
  Using cached llama_index_program_openai-0.3.2-py3-none-any.whl.metadata (473 bytes)
  Using cached llama_index_question_gen_openai-0.3.1-py3-none-any.whl.metadata (492 bytes)
  Using cached llama_index_readers_file-0.4.9-py3-none-any.whl.metadata (5.2 kB)
  Using cached llama_index_readers_llama_parse-0.4.0-py3-none-any.whl.metadata (3.6 kB)
  Using cached python_dotenv-1.1.1-py3-none-any.whl.metadata (24 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 282.1/282.1 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━

In [7]:
pip show llama-index llama-index-core llama-index-embeddings-google-genai


Name: llama-index
Version: 0.12.46
Summary: Interface between LLMs and your data
Home-page: https://llamaindex.ai
Author: 
Author-email: Jerry Liu <jerry@llamaindex.ai>
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: llama-index-agent-openai, llama-index-cli, llama-index-core, llama-index-embeddings-openai, llama-index-indices-managed-llama-cloud, llama-index-llms-openai, llama-index-multi-modal-llms-openai, llama-index-program-openai, llama-index-question-gen-openai, llama-index-readers-file, llama-index-readers-llama-parse, nltk
Required-by: 
---
Name: llama-index-core
Version: 0.12.46
Summary: Interface between LLMs and your data
Home-page: https://llamaindex.ai
Author: 
Author-email: Jerry Liu <jerry@llamaindex.ai>
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: aiohttp, aiosqlite, banks, dataclasses-json, deprecated, dirtyjson, filetype, fsspec, httpx, llama-index-workflows, nest-asyncio, networkx, nltk, numpy, pillow, pydantic, pyyaml, 

## Step 3: Set up your Gemini API key

In [8]:
# Set up your Gemini API key
import os
from getpass import getpass

# Securely input your API key
GEMINI_API_KEY = getpass('Enter your Gemini API key: ')
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

# Create .env file with the API key
with open('.env', 'w') as f:
    f.write(f"GEMINI_API_KEY={GEMINI_API_KEY}\n")
    f.write("CHROMA_DB_DIRECTORY=./chroma_db\n")

Enter your Gemini API key: ··········


## Step 4: Create a version-flexible chatbot implementation

This version handles different import structures in different versions of LlamaIndex

In [9]:
%%writefile colab_chatbot_flexible.py
import os
import sys
import logging
import google.generativeai as genai
from dotenv import load_dotenv
import chromadb

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Load environment variables
load_dotenv()

# Directory paths
DATA_DIR = "./data"
CHROMA_DB_DIRECTORY = "./chroma_db"

class GeminiChatbot:
    def __init__(self):
        # Get Gemini API key
        self.api_key = os.getenv("GEMINI_API_KEY")
        if not self.api_key:
            print("GEMINI_API_KEY not found in environment. Please set it first.")
            self.api_key = input("Enter your Gemini API key: ")
            os.environ["GEMINI_API_KEY"] = self.api_key

        # Configure Gemini
        genai.configure(api_key=self.api_key)
        self.model = genai.GenerativeModel('gemini-pro')
        self.index = None

    def load_documents(self):
        """Load documents from the data directory"""
        if not os.path.exists(DATA_DIR):
            os.makedirs(DATA_DIR)
            print(f"Created data directory at {DATA_DIR}")

        if not os.listdir(DATA_DIR):
            print("Data directory is empty. Please add some documents.")
            return False

        return True

    def build_index(self):
        """Build vector index from documents"""
        try:
            # Dynamic imports to handle different versions of llama-index
            try:
                # Try importing from llama_index (newer versions)
                from llama_index import VectorStoreIndex, SimpleDirectoryReader, StorageContext
                from llama_index.node_parser import SentenceSplitter
                from llama_index.vector_stores.chroma import ChromaVectorStore
                print("Using llama_index package")
            except ImportError:
                # Try importing from llama_index.core (older or different versions)
                from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
                from llama_index.core.node_parser import SentenceSplitter
                try:
                    from llama_index.core.vector_stores.chroma import ChromaVectorStore
                    print("Using llama_index.core package")
                except ImportError:
                    from llama_index.vector_stores.chroma import ChromaVectorStore
                    print("Using mixed llama_index imports")

            # Load documents
            documents = SimpleDirectoryReader(DATA_DIR).load_data()

            # Create sentence splitter for text chunking
            text_splitter = SentenceSplitter(
                chunk_size=1000,
                chunk_overlap=200
            )

            # Set up Chroma client
            if not os.path.exists(CHROMA_DB_DIRECTORY):
                os.makedirs(CHROMA_DB_DIRECTORY)

            chroma_client = chromadb.PersistentClient(path=CHROMA_DB_DIRECTORY)
            chroma_collection = chroma_client.get_or_create_collection("documents")
            vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
            storage_context = StorageContext.from_defaults(vector_store=vector_store)

            # Build index
            try:
                self.index = VectorStoreIndex.from_documents(
                    documents,
                    storage_context=storage_context,
                    transformations=[text_splitter],
                )
            except TypeError:
                # Try alternative approach if transformations parameter doesn't work
                self.index = VectorStoreIndex.from_documents(
                    documents,
                    storage_context=storage_context,
                    node_parser=text_splitter,
                )

            print("Index built successfully!")
            return True
        except Exception as e:
            print(f"Error building index: {str(e)}")
            import traceback
            traceback.print_exc()
            return False

    def query(self, query_text):
        """Query the index with a natural language query"""
        if not self.index:
            print("Index not built yet. Please build the index first.")
            return {
                "answer": "Index not built yet. Please build the index first.",
                "sources": []
            }

        try:
            # Create query engine
            query_engine = self.index.as_query_engine()

            # Execute query
            response = query_engine.query(query_text)

            # Extract source information
            sources = []
            if hasattr(response, 'source_nodes'):
                for node in response.source_nodes:
                    source = {
                        "text": node.node.get_content(),
                        "metadata": node.node.metadata,
                        "score": node.score if hasattr(node, 'score') else None
                    }
                    sources.append(source)

            return {
                "answer": str(response),
                "sources": sources
            }
        except Exception as e:
            print(f"Error during query: {str(e)}")
            import traceback
            traceback.print_exc()
            return {
                "answer": f"Error during query: {str(e)}",
                "sources": []
            }

    def chat_interface(self):
        """Simple chat interface for testing"""
        print("\n===== Gemini Chatbot =====")
        print("Type 'exit' to quit")
        print("========================\n")

        while True:
            query = input("\nYou: ")
            if query.lower() in ['exit', 'quit', 'q']:
                break

            response = self.query(query)
            print(f"\nBot: {response['answer']}")

            # Print sources
            if response['sources']:
                print("\nSources:")
                for i, source in enumerate(response['sources'][:2], 1):
                    source_name = source.get('metadata', {}).get('source', f"Source {i}")
                    print(f"- {source_name}")

if __name__ == "__main__":
    print("Initializing chatbot...")
    chatbot = GeminiChatbot()

    print("Loading documents...")
    if chatbot.load_documents():
        print("Building index...")
        if chatbot.build_index():
            print("Ready to chat!")
            chatbot.chat_interface()

Overwriting colab_chatbot_flexible.py


## Step 5: Upload custom content (Optional)

In [10]:
# Upload your own content files (optional)
from google.colab import files
import os

# Create data directory if it doesn't exist
if not os.path.exists('data'):
    os.makedirs('data')

# Let user upload files
print("Upload your content files (optional). Skip this if you want to use the sample FAQ.")
try:
    uploaded = files.upload()

    # Save uploaded files to the data directory
    for filename, content in uploaded.items():
        with open(f'data/{filename}', 'wb') as f:
            f.write(content)
        print(f"Saved {filename} to data directory")
except Exception as e:
    print(f"No files uploaded or error occurred: {e}")

Upload your content files (optional). Skip this if you want to use the sample FAQ.


## Step 6: Run the chatbot

In [15]:
# Run the command-line chatbot
!python colab_chatbot_flexible.py

Initializing chatbot...
Loading documents...
Building index...
Using mixed llama_index imports
INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
Error building index: 
******
Could not load OpenAI embedding model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

Consider using embed_model='local'.
Visit our documentation for more embedding options: https://docs.llamaindex.ai/en/stable/module_guides/models/embeddings.html#modules
******
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/llama_index/core/embeddings/utils.py", line 59, in resolve_embed_model
    val

In [12]:
 !pip install chromadb

  Using cached chromadb-1.0.15-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [14]:
pip install llama-index-vector-stores-chroma


In [ ]:
# Create a sample document in the data directory
!mkdir -p data
%%writefile data/sample_faq.md
# Sample FAQ

## What is this chatbot?
This is a RAG-based chatbot using Google Gemini API.

## What is RAG?
RAG stands for Retrieval-Augmented Generation, which enhances LLM responses with retrieved information.

## How does it work?
It uses vector embeddings to find relevant information and then generates responses based on that information.

In [ ]:
# Run the chatbot in the same Colab notebook
%run colab_chatbot_flexible.py

In [ ]:
import os
from google.colab import userdata

# Try to get the API key from Colab secrets
try:
  GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
except:
  # If not in secrets, ask user for it
  GEMINI_API_KEY = input('Enter your Gemini API key: ')

os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

# Create .env file with the API key
with open('.env', 'w') as f:
    f.write(f"GEMINI_API_KEY={GEMINI_API_KEY}\n")

print("API key set successfully!")

## Step 7: Create Streamlit Interface (Optional)

In [ ]:
%%writefile colab_streamlit_app.py
import streamlit as st
import os
import google.generativeai as genai
from dotenv import load_dotenv
import chromadb

# Load environment variables
load_dotenv()

# Configure Gemini
api_key = os.getenv("GEMINI_API_KEY")
if api_key:
    genai.configure(api_key=api_key)

# Directory paths
DATA_DIR = "./data"
CHROMA_DB_DIRECTORY = "./chroma_db"

# Initialize session state
if 'index' not in st.session_state:
    st.session_state.index = None
if 'messages' not in st.session_state:
    st.session_state.messages = []

def build_index():
    """Build or load the vector index"""
    try:
        # Dynamic imports to handle different versions of llama-index
        try:
            # Try importing from llama_index (newer versions)
            from llama_index import VectorStoreIndex, SimpleDirectoryReader, StorageContext
            from llama_index.node_parser import SentenceSplitter
            from llama_index.vector_stores.chroma import ChromaVectorStore
            st.info("Using llama_index package")
        except ImportError:
            # Try importing from llama_index.core (older or different versions)
            from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
            from llama_index.core.node_parser import SentenceSplitter
            try:
                from llama_index.core.vector_stores.chroma import ChromaVectorStore
                st.info("Using llama_index.core package")
            except ImportError:
                from llama_index.vector_stores.chroma import ChromaVectorStore
                st.info("Using mixed llama_index imports")

        if not os.path.exists(DATA_DIR):
            os.makedirs(DATA_DIR)

        # Load documents
        documents = SimpleDirectoryReader(DATA_DIR).load_data()

        # Create sentence splitter for text chunking
        text_splitter = SentenceSplitter(
            chunk_size=1000,
            chunk_overlap=200
        )

        # Set up Chroma client
        if not os.path.exists(CHROMA_DB_DIRECTORY):
            os.makedirs(CHROMA_DB_DIRECTORY)

        chroma_client = chromadb.PersistentClient(path=CHROMA_DB_DIRECTORY)
        chroma_collection = chroma_client.get_or_create_collection("documents")
        vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
        storage_context = StorageContext.from_defaults(vector_store=vector_store)

        # Build index
        try:
            index = VectorStoreIndex.from_documents(
                documents,
                storage_context=storage_context,
                transformations=[text_splitter],
            )
        except TypeError:
            # Try alternative approach if transformations parameter doesn't work
            index = VectorStoreIndex.from_documents(
                documents,
                storage_context=storage_context,
                node_parser=text_splitter,
            )

        return index
    except Exception as e:
        st.error(f"Error building index: {str(e)}")
        import traceback
        st.error(traceback.format_exc())
        return None

def query_index(query_text):
    """Query the index with a natural language query"""
    if not st.session_state.index:
        return {
            "answer": "Index not built yet. Please build the index first.",
            "sources": []
        }

    try:
        # Create query engine
        query_engine = st.session_state.index.as_query_engine()

        # Execute query
        response = query_engine.query(query_text)

        # Extract source information
        sources = []
        if hasattr(response, 'source_nodes'):
            for node in response.source_nodes:
                source = {
                    "text": node.node.get_content(),
                    "metadata": node.node.metadata,
                    "score": node.score if hasattr(node, 'score') else None
                }
                sources.append(source)

        return {
            "answer": str(response),
            "sources": sources
        }
    except Exception as e:
        import traceback
        st.error(traceback.format_exc())
        return {
            "answer": f"Error during query: {str(e)}",
            "sources": []
        }

# App title
st.title("🤖 Gemini Chatbot")

# Sidebar
with st.sidebar:
    st.title("Settings")

    # API key input
    if not api_key:
        new_api_key = st.text_input("Enter Gemini API Key", type="password")
        if new_api_key:
            genai.configure(api_key=new_api_key)
            os.environ["GEMINI_API_KEY"] = new_api_key
            with open('.env', 'w') as f:
                f.write(f"GEMINI_API_KEY={new_api_key}\n")
                f.write("CHROMA_DB_DIRECTORY=./chroma_db\n")
            st.success("API key set!")
            api_key = new_api_key

    # Build index button
    if st.button("Build/Rebuild Index"):
        with st.spinner("Building index..."):
            st.session_state.index = build_index()
            if st.session_state.index:
                st.success("Index built successfully!")
            else:
                st.error("Failed to build index")

    # Show data directory contents
    st.subheader("Data Directory")
    if os.path.exists(DATA_DIR):
        files = os.listdir(DATA_DIR)
        if files:
            st.write(f"Found {len(files)} files:")
            for file in files:
                st.write(f"- {file}")
        else:
            st.write("No files found in data directory")
    else:
        st.write("Data directory does not exist")

    # Clear chat history
    if st.button("Clear Chat History"):
        st.session_state.messages = []
        st.success("Chat history cleared")

# Initialize or load index
if st.session_state.index is None:
    if os.path.exists(CHROMA_DB_DIRECTORY):
        with st.spinner("Loading existing index..."):
            st.session_state.index = build_index()
            if st.session_state.index:
                st.success("Index loaded successfully!")
    else:
        st.info("No index found. Please build the index using the sidebar button.")

# Display chat messages
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

        # Show sources if available
        if message["role"] == "assistant" and "sources" in message:
            with st.expander("View Sources"):
                for i, source in enumerate(message["sources"], 1):
                    source_name = source.get('metadata', {}).get('source', f"Source {i}")
                    st.markdown(f"**{source_name}**")
                    text = source.get('text', '')
                    if text:
                        st.text(text[:200] + "..." if len(text) > 200 else text)

# Chat input
if prompt := st.chat_input("Ask me anything..."):
    # Check if API key is set
    if not api_key:
        st.error("Please set your Gemini API key in the sidebar first.")
        st.stop()

    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})

    # Display user message
    with st.chat_message("user"):
        st.markdown(prompt)

    # Generate and display assistant response
    with st.chat_message("assistant"):
        with st.spinner("Thinking..."):
            if st.session_state.index is None:
                response_content = "Please build the index first using the sidebar button."
                st.markdown(response_content)
                st.session_state.messages.append({
                    "role": "assistant",
                    "content": response_content
                })
            else:
                try:
                    response = query_index(prompt)
                    st.markdown(response["answer"])

                    # Add assistant message to chat history
                    st.session_state.messages.append({
                        "role": "assistant",
                        "content": response["answer"],
                        "sources": response["sources"]
                    })

                    # Show sources if available
                    if response["sources"]:
                        with st.expander("View Sources"):
                            for i, source in enumerate(response["sources"], 1):
                                source_name = source.get('metadata', {}).get('source', f"Source {i}")
                                st.markdown(f"**{source_name}**")
                                text = source.get('text', '')
                                if text:
                                    st.text(text[:200] + "..." if len(text) > 200 else text)
                except Exception as e:
                    error_msg = f"Error: {str(e)}"
                    st.error(error_msg)
                    st.session_state.messages.append({
                        "role": "assistant",
                        "content": error_msg
                    })

## Step 8: Create the Streamlit interface for Google Colab

In [ ]:
%%writefile colab_streamlit_app.py
#!/usr/bin/env python3
"""
Streamlit web interface for the chatbot - Google Colab version
"""
import streamlit as st
import logging
import sys
import os
import google.generativeai as genai
from dotenv import load_dotenv
import chromadb

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Load environment variables
load_dotenv()

# Directory paths
DATA_DIR = "./data"
CHROMA_DB_DIRECTORY = "./chroma_db"

class GeminiChatbot:
    def __init__(self):
        # Get Gemini API key
        self.api_key = os.getenv("GEMINI_API_KEY")
        if not self.api_key:
            print("GEMINI_API_KEY not found in environment. Please set it first.")
            self.api_key = st.text_input("Enter your Gemini API key:", type="password")
            if not self.api_key:
                st.error("Gemini API key is required to continue.")
                st.stop()
            os.environ["GEMINI_API_KEY"] = self.api_key

        # Configure Gemini
        genai.configure(api_key=self.api_key)
        self.model = genai.GenerativeModel('gemini-pro')
        self.index = None

    def load_documents(self):
        """Load documents from the data directory"""
        if not os.path.exists(DATA_DIR):
            os.makedirs(DATA_DIR)
            st.info(f"Created data directory at {DATA_DIR}")

        if not os.listdir(DATA_DIR):
            st.warning("Data directory is empty. Please add some documents.")
            return False

        return True

    def build_index(self):
        """Build vector index from documents"""
        try:
            # Dynamic imports to handle different versions of llama-index
            try:
                # Try importing from llama_index (newer versions)
                from llama_index import VectorStoreIndex, SimpleDirectoryReader, StorageContext
                from llama_index.node_parser import SentenceSplitter
                from llama_index.vector_stores.chroma import ChromaVectorStore
                st.info("Using llama_index package")
            except ImportError:
                # Try importing from llama_index.core (older or different versions)
                from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
                from llama_index.core.node_parser import SentenceSplitter
                try:
                    from llama_index.core.vector_stores.chroma import ChromaVectorStore
                    st.info("Using llama_index.core package")
                except ImportError:
                    from llama_index.vector_stores.chroma import ChromaVectorStore
                    st.info("Using mixed llama_index imports")

            # Load documents
            documents = SimpleDirectoryReader(DATA_DIR).load_data()

            # Create sentence splitter for text chunking
            text_splitter = SentenceSplitter(
                chunk_size=1000,
                chunk_overlap=200
            )

            # Set up Chroma client
            if not os.path.exists(CHROMA_DB_DIRECTORY):
                os.makedirs(CHROMA_DB_DIRECTORY)

            chroma_client = chromadb.PersistentClient(path=CHROMA_DB_DIRECTORY)
            chroma_collection = chroma_client.get_or_create_collection("documents")
            vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
            storage_context = StorageContext.from_defaults(vector_store=vector_store)

            # Build index with error handling for different parameter names
            try:
                self.index = VectorStoreIndex.from_documents(
                    documents,
                    storage_context=storage_context,
                    transformations=[text_splitter],
                )
            except TypeError:
                # Try alternative approach if transformations parameter doesn't work
                self.index = VectorStoreIndex.from_documents(
                    documents,
                    storage_context=storage_context,
                    node_parser=text_splitter,
                )

            st.success("Index built successfully!")
            return True
        except Exception as e:
            st.error(f"Error building index: {str(e)}")
            import traceback
            traceback.print_exc()
            return False

    def query(self, query_text):
        """Query the index with a natural language query"""
        if not self.index:
            st.error("Index not built yet. Please build the index first.")
            return {
                "answer": "Index not built yet. Please build the index first.",
                "sources": []
            }

        try:
            # Create query engine
            query_engine = self.index.as_query_engine()

            # Execute query
            response = query_engine.query(query_text)

            # Extract source information
            sources = []
            if hasattr(response, 'source_nodes'):
                for node in response.source_nodes:
                    source = {
                        "text": node.node.get_content(),
                        "metadata": node.node.metadata,
                        "score": node.score if hasattr(node, 'score') else None
                    }
                    sources.append(source)

            return {
                "answer": str(response),
                "sources": sources
            }
        except Exception as e:
            st.error(f"Error during query: {str(e)}")
            import traceback
            traceback.print_exc()
            return {
                "answer": f"Error during query: {str(e)}",
                "sources": []
            }

def init_session_state():
    """Initialize session state variables"""
    if 'messages' not in st.session_state:
        st.session_state.messages = []
    if 'chatbot' not in st.session_state:
        st.session_state.chatbot = None

def main():
    # Set page config
    st.set_page_config(
        page_title="Gemini Chatbot for Google Colab",
        page_icon="🤖",
        layout="wide"
    )

    # Initialize session state
    init_session_state()

    # Main title
    st.title("🤖 Gemini Chatbot for Google Colab")

    # Sidebar
    with st.sidebar:
        st.title("Chatbot Settings")

        # Initialize chatbot if not already done
        if st.session_state.chatbot is None:
            st.session_state.chatbot = GeminiChatbot()

        # Check data directory status
        data_dir_status = "✅ Available" if os.path.exists(DATA_DIR) and os.listdir(DATA_DIR) else "❌ Empty"
        st.info(f"Data Directory: {data_dir_status}")

        # Check index status
        index_status = "✅ Available" if os.path.exists(CHROMA_DB_DIRECTORY) else "❌ Not Built"
        st.info(f"Vector Index: {index_status}")

        # Upload documents
        with st.expander("Upload Documents"):
            uploaded_file = st.file_uploader("Upload a document", type=["txt", "md", "pdf"])
            if uploaded_file is not None:
                if not os.path.exists(DATA_DIR):
                    os.makedirs(DATA_DIR)

                file_path = os.path.join(DATA_DIR, uploaded_file.name)
                with open(file_path, "wb") as f:
                    f.write(uploaded_file.getvalue())
                st.success(f"File '{uploaded_file.name}' uploaded successfully")

        # Build index button
        if st.button("Build/Rebuild Vector Index"):
            if st.session_state.chatbot.load_documents():
                with st.spinner("Building vector index..."):
                    if st.session_state.chatbot.build_index():
                        st.success("Vector index built successfully!")
                    else:
                        st.error("Failed to build vector index.")
            else:
                st.error("No documents found. Please upload some documents first.")

    # Display chat messages
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

            # Show sources if available
            if message["role"] == "assistant" and "sources" in message and message["sources"]:
                with st.expander("View Sources"):
                    for i, source in enumerate(message["sources"], 1):
                        source_name = source.get('metadata', {}).get('source', f"Source {i}")
                        st.markdown(f"**Document: {source_name}**")
                        text = source.get('text', '')
                        if text:
                            st.text(text[:200] + "..." if len(text) > 200 else text)

    # Chat input
    if prompt := st.chat_input("Ask me anything..."):
        # Add user message to chat history
        st.session_state.messages.append({"role": "user", "content": prompt})

        # Display user message
        with st.chat_message("user"):
            st.markdown(prompt)

        # Check if index is built
        if not st.session_state.chatbot.index:
            if not os.path.exists(CHROMA_DB_DIRECTORY) or not os.listdir(CHROMA_DB_DIRECTORY):
                # Need to build index first
                if st.session_state.chatbot.load_documents():
                    with st.spinner("Building index for the first time..."):
                        st.session_state.chatbot.build_index()
                else:
                    st.error("No documents found. Please upload some documents first.")
                    st.session_state.messages.append({
                        "role": "assistant",
                        "content": "I need some documents to learn from. Please upload documents using the sidebar.",
                        "sources": []
                    })
                    st.stop()
            else:
                # Index exists but not loaded
                with st.spinner("Loading existing index..."):
                    st.session_state.chatbot.build_index()

        # Generate and display assistant response
        with st.chat_message("assistant"):
            message_placeholder = st.empty()
            with st.spinner("Thinking..."):
                response = st.session_state.chatbot.query(prompt)
                message_placeholder.markdown(response["answer"])

                # Add assistant message to chat history
                st.session_state.messages.append({
                    "role": "assistant",
                    "content": response["answer"],
                    "sources": response["sources"]
                })

                # Show sources if available
                if response["sources"]:
                    with st.expander("View Sources"):
                        for i, source in enumerate(response["sources"], 1):
                            source_name = source.get('metadata', {}).get('source', f"Source {i}")
                            st.markdown(f"**Document: {source_name}**")
                            text = source.get('text', '')
                            if text:
                                st.text(text[:200] + "..." if len(text) > 200 else text)

if __name__ == "__main__":
    main()

## Step 9: Run Streamlit Interface (Optional)

In [ ]:
# Install ngrok and pyngrok if you want to run the Streamlit app
!pip install pyngrok

# Run Streamlit app using ngrok
from pyngrok import ngrok
import os

# Set up ngrok authentication (optional, needed for better reliability)
# Get your token from https://dashboard.ngrok.com/get-started/your-authtoken
NGROK_AUTH_TOKEN = input("Enter your ngrok auth token (optional, press Enter to skip): ")
if NGROK_AUTH_TOKEN:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Start ngrok tunnel to Streamlit
public_url = ngrok.connect(8501)
print(f"\n\nStreamlit app is available at: {public_url}\n\n")

# Run Streamlit app
!streamlit run colab_streamlit_app.py &

## Step 10: Clean up (Run this when you're done)

In [ ]:
# Clean up
!pkill -f streamlit  # Stop Streamlit if running
ngrok.kill()  # Stop ngrok if running